### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [ ]:
#Importing classes from the Jinja2 library to load and render templates.
import json
import os
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError
from pathlib import Path


In [ ]:
os.getcwd()

In [ ]:
current_dir = Path(os.getcwd())

MEMBERS_DIR_PATH = current_dir / '../../group-data/members/'
WEBSITE_DATA_PATH = current_dir / '../../group-data/website_data/'
CONTENT_DIR_PATH = current_dir / '../../group-data/website_data/content/'


In [ ]:
#Function for creating proper html file names
def page_link(a):
    if ' ' in a:
        return a.replace(' ', '_')
    else:
        return a

In [ ]:
#Loading members
member_records = []

for member_dir in MEMBERS_DIR_PATH.glob('*'):
   
    if member_dir.name.startswith('.'):
        continue
    
    member_record = json.load(open(member_dir / 'info.json'))
    member_json_dir = member_dir / 'jsons'
    experiences_education_df = []
    if (member_experiences := (member_json_dir / 'experiences.json')).exists():
        experiences = json.load(open(member_experiences))
        #current_position = {f"exp_{key}": value for key, value in experiences[0].items()}
        experiences_education_df += experiences
 
        
    if (member_education := (member_json_dir / 'education.json')).exists():
        education = json.load(open(member_education))
        experiences_education_df += education
        #current_position = {f"edu_{key}": value for key, value in education[0].items()}
       

    if (member_projects := (member_json_dir / 'projects.json')).exists():
        current_position = json.load(open(member_projects))[0]
        member_record.update(current_position)
    
    if (member_links := (member_json_dir / 'social_links.json')).exists():
        social_links = json.load(open(member_links))
        member_record.update(social_links)
    
    member_records.append(member_record)
    break

people_page_df = pd.DataFrame.from_records(member_records)
experiences_education_df = pd.DataFrame.from_records(experiences_education_df)


In [ ]:
experiences_education_df.sort_values('start_date')

In [ ]:
"""
def process_education_and_experiences(df):
    results = []

    for index, row in df.iterrows():
        degree_key = None
        end_date_present = False
        role = None


        if 'degree' in row:
            degree_key = row['degree']


        if 'end_date' in row and pd.notna(row['end_date']):
            end_date_present = True

 
        if 'role' in row:
            role = row['role']


        if degree_key == 'PhD' or degree_key == 'Masters':
            degree_type = 'Graduate'
        elif degree_key == 'Bachelors':
            degree_type = 'Undergraduate'
        else:
            degree_type = 'Unknown'


        if end_date_present:
            status = 'Alumni'
        else:
            status = 'Current Member'

        result = {
            'Degree Type': degree_type,
            'Member Status': status,
            'Role': role
        }

        results.append(result)

    return pd.DataFrame(results)


result_df = process_education_and_experiences(experiences_education_df)


print(result_df)
"""

In [ ]:
#Func to display current role
"""
def func_edu_ex_df(data_df):


    experiences_df = data_df
    is_member = experiences_df['end_date'].isna().any()
    top_experience_role = experiences_df.iloc[0]['role']
    
    if is_member:
        print(f"{top_experience_role}: Current member")
    else:
        print(f"{top_experience_role} Alumni")

func_edu_ex_df(experiences_education_df)
"""

In [ ]:
#Func to return category.
def func_edu_ex_df(data_df):


    top_education_degree = data_df.iloc[0]['degree']
    

    top_experience_role = data_df.iloc[0]['role']


    if top_education_degree == 'Bachelors':
        return 'Undergraduate'
    elif top_education_degree in ['Masters', 'PhD']:
        return 'Graduate Student'
    

    if top_experience_role in ['Visiting Researcher', 'Postdoctoral Researcher']:
        return 'Postdoctoral researcher'
    elif 'Assistant Professor' in top_experience_role or 'Professor' in top_experience_role:
        return 'Professor'
    

    return 'somethin else' # for 


result = func_edu_ex_df(experiences_education_df)
print(result)


In [ ]:
people_page_df

In [ ]:
# Creating pandas dataframes for Present Members and Alumni
"""

degree_condition = people_page_df['edu_degree'].str.strip().str.lower().isin(['bachelors', 'masters', 'phd', 'postdoc'])

end_date_condition = (people_page_df['edu_end_date'].isna()) | (people_page_df['exp_end_date'].isna())


people_page_df['edu_institution'] = people_page_df['edu_institution'].str.strip()
people_page_df['exp_institution'] = people_page_df['exp_institution'].str.strip()

institution_condition = (
    (people_page_df['edu_institution'].str.lower() == 'michigan state university') |
     (people_page_df['edu_institution'].str.lower() == 'msu')|
    (people_page_df['exp_institution'].str.lower() == 'michigan state university') |
        (people_page_df['exp_institution'].str.lower() == 'msu') 
)

final_filter = degree_condition & end_date_condition & institution_condition


people_present_df = people_page_df[final_filter]
people_alumni_df = people_page_df[~final_filter]
"""

In [ ]:
#people_alumni_df[['first_name','edu_degree','edu_institution', 'edu_end_date' , 'exp_institution','exp_end_date']]
#people_alumni_df[['first_name','edu_degree','edu_institution', 'edu_end_date' , 'exp_institution','exp_end_date']]


In [ ]:



website_data_files = {
    'contact': 'contact.json',
    'general': 'general.json',
    'research': 'research.json',
    'homepage': 'homepage.json',
    'support': 'support.json'
}


contact, general, research, homepage, support = [{}] * 5


for var_name, webpage_json in website_data_files.items():
    file_path = WEBSITE_DATA_PATH / webpage_json
    if file_path.exists():
        with open(file_path, 'r') as f:
            locals()[var_name] = json.load(f)


In [ ]:
contact_df = pd.DataFrame.from_records([contact])
general_df = pd.DataFrame.from_records([general])
homepage_df = pd.DataFrame.from_records([homepage])
research_df = pd.DataFrame.from_records([research])
support_df = pd.DataFrame.from_records([support])


In [ ]:

content_dfs = []  
for json_file in CONTENT_DIR_PATH.glob('*.json'):
    with open(json_file, 'r') as file:
        data = json.load(file)
        
       
        df = pd.json_normalize(data)
        
        content_dfs.append(df)


content_df = pd.concat(content_dfs, ignore_index=True, sort=False)




In [ ]:
#Creating an instance of the Environment class that looks for templates. Page_link is set to the global variable so that it can be accessed by all templates
environment = Environment(loader=FileSystemLoader('templates/'),extensions=['jinja2.ext.loopcontrols'])
environment.globals['page_link'] = page_link
environment.globals['ContentData'] = content_df

In [ ]:
research_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('research' in tag for tag in x))]
research_content = research_content_unsorted.groupby('category').apply(lambda x: x.sort_values('article_date', ascending=False)).reset_index(drop=True)

In [ ]:
news_content_unsorted = content_df[content_df['tags'].apply(lambda x: any('news' in tag for tag in x))]
news_content = news_content_unsorted.sort_values(by="article_date", ascending=False)

In [ ]:
latest_content_df = pd.DataFrame()

for category in content_df.category.unique():
    latest_data = pd.Series(content_df[content_df.category == category].iloc[0])
    latest_content_df = latest_content_df._append(latest_data, ignore_index=True)

latest_content_df['article_date'] = pd.to_datetime(latest_content_df['article_date'], format='%m-%d-%Y')
latest_content_df = latest_content_df.sort_values(by='article_date', ascending=False)

##### Homepage

In [ ]:
homepage_template = environment.get_template('homepage.html.j2')

In [ ]:
general_dict = general_df.iloc[0].to_dict()
homepage_dict = homepage_df.iloc[0].to_dict()


In [ ]:
homepage_content = homepage_template.render(general=general_dict,
                                            homepage=homepage_dict,
                                            recent_content=latest_content_df.to_dict(orient='records'))


In [ ]:
with open('../kerzendorf-group.github.io/index.html', mode='w', encoding='utf-8') as Homepage:
    Homepage.write(homepage_content)

##### People Page

In [ ]:
templates_dir_path = current_dir / '../templates/'
environment = Environment(loader=FileSystemLoader(str(templates_dir_path)))
people_template = environment.get_template("people.html.j2")


In [ ]:
people_template = environment.get_template("people.html.j2")

In [ ]:
environment.globals['page_link'] = page_link
people_content = people_template.render(general=general_dict, present_members=people_page_df, result=result)


In [ ]:
with open("../../kerzendorf-group.github.io/People.html", mode="w", encoding="utf-8") as people:
    people.write(people_content)

##### Individual People Page

In [ ]:
ind_person_template = environment.get_template("individual_person.html.j2")

In [ ]:
for person in people_df['id']:
            filename = f"../kerzendorf-group.github.io/members/{ person }/{ person }.html"
            ind_person_content = ind_person_template.render(general=data["general"], 
                                                            member_id=person, 
                                                            content=content_df.to_dict(orient='records'))
            with open(filename, mode="w", encoding="utf-8") as page:
                page.write(ind_person_content)

##### Research Page

In [ ]:
research_template = environment.get_template("research.html.j2")

In [ ]:
main_page_research_content = research_template.render(general=data["general"],
                                            content=research_content)

In [ ]:
with open("../kerzendorf-group.github.io/Research.html", mode="w", encoding="utf-8") as research:
        research.write(main_page_research_content)

In [ ]:
sub_research_template = environment.get_template("sub_research_frontpage.html.j2")

In [ ]:
for category in content_df.loc[content_df.category != "News", "category"].unique():
        sub_research_content = sub_research_template.render(general=data["general"], 
                                                            research_general=data["research"], 
                                                            content = research_content,
                                                            category = category
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"../kerzendorf-group.github.io/sub_research/{page_link(category.lower())}.html", mode="w", encoding="utf-8") as sub_research:
            sub_research.write(sub_research_content)

##### Individual Research Page

In [ ]:
template_no_twitter = environment.get_template("research_page_no_twitter.html.j2")

In [ ]:
for ind_research_keys, ind_research_values in research_content.iterrows():
    if "news" not in ind_research_values.category.lower():
        ind_research_content = template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_research_values
                                                          
                                                            )
        folder_path = f"../kerzendorf-group.github.io/sub_research/{page_link(ind_research_values.category.lower())}"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_research_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_research_page:
            ind_research_page.write(ind_research_content)

##### News Page

In [ ]:
news_content

In [ ]:
news_template = environment.get_template("news.html.j2")

In [ ]:
news_page_content = news_template.render(general=data["general"],
                                         content=news_content,
                                         member_ids=people_df['id'],
                                         nonmem_ids = non_mem_df['id'],
                                         category="News")

In [ ]:
with open("../kerzendorf-group.github.io/News.html", mode="w", encoding="utf-8") as news:
        news.write(news_page_content)

##### Individual News Pages

In [ ]:
news_template_no_twitter = environment.get_template("news_page_no_twitter.html.j2")
#news_template_twitter = environment.get_template("news_page_twitter.html.j2")

In [ ]:
for ind_news_keys, ind_news_values in news_content.iterrows():
        ind_news_content = news_template_no_twitter.render(general=data["general"], 
                                                          member_ids = people_df['id'],
                                                          nonmem_ids = non_mem_df['id'],
                                                          content = ind_news_values
                                                            )
        folder_path = f"../kerzendorf-group.github.io/news/"
        os.makedirs(folder_path, exist_ok=True)
        with open(f"{ folder_path }/{page_link(ind_news_values.article_id.lower())}.html", mode="w", encoding="utf-8") as ind_news_page:
            ind_news_page.write(ind_news_content)

##### Support Page

In [ ]:
support_template = environment.get_template('support.html.j2')

In [ ]:
support_content = support_template.render(general=data["general"], support=data["support"])

In [ ]:
with open('../kerzendorf-group.github.io/Support.html', mode='w', encoding='utf-8') as support:
    support.write(support_content)

##### Contact

In [ ]:
contact_template = environment.get_template('contact.html.j2')

In [ ]:
contact_content = contact_template.render(general=data["general"], contact=data["contact"])

In [ ]:
with open('../kerzendorf-group.github.io/Contact.html', mode='w', encoding='utf-8') as contact:
    contact.write(contact_content)